In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split


from interpret.glassbox import LogisticRegression
from interpret import show

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../Processing/')

from preprocessing import Preprocess_English_Sentence,Preprocess_French_Sentence
from sklearn.feature_extraction.text import CountVectorizer

import wordcloud

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\theov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\theov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\theov\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\theov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
with open("../Processing/vocab.txt", "r",encoding="utf-8")as f :
    vocab = f.read().split("\n")
    vectorizer = CountVectorizer(max_features=3000,vocabulary=vocab,stop_words="english",binary=True)


In [3]:
import os
os.listdir("../Processing/processed_files/wattpad/")

['.ipynb_checkpoints',
 'crime-encoded.csv',
 'crime-preprocessed.json',
 'cyberpunk-encoded.csv',
 'cyberpunk-preprocessed.json',
 'cyberpunk-w-encoding.json',
 'dystopie-encoded.csv',
 'dystopie-encoded.json',
 'dystopie-preprocessed.json',
 'fantasy-encoded.csv',
 'fantasy-preprocessed.json',
 'horror-encoded.csv',
 'horror-encoded.json',
 'horror-preprocessed.json',
 'humour-encoded.csv',
 'humour-encoded.json',
 'humour-preprocessed.json',
 'romance-encoded-clean.json',
 'romance-encoded.csv',
 'romance-preprocessed.json',
 'science-fantasy-encoded.csv',
 'science-fantasy-encoded.json',
 'science-fantasy-preprocessed.json',
 'space-encoded.csv',
 'space-encoded.json',
 'space-preprocessed.json',
 'steampunk-encoded.csv',
 'steampunk-encoded.json',
 'steampunk-preprocessed.json',
 'thriller-encoded.csv',
 'thriller-encoded.json',
 'thriller-preprocessed.json',
 'vampire-encoded.csv',
 'vampire-encoded.json',
 'vampire-preprocessed.json']

In [4]:
cyberpunk = pd.read_json("../Processing/processed_files/wattpad/cyberpunk-preprocessed.json")

In [5]:
import dateutil.parser as dparser
cyberpunk["year_publication"] = cyberpunk["first_update"].apply(lambda x : dparser.parse(x,fuzzy=True).year)

In [6]:
cyberpunk.columns

Index(['story_title', 'story_id', 'reads', 'votes', 'chapterNb',
       'first_update', 'last_update', 'story_text', 'preprocess_story_text',
       'language', 'year_publication'],
      dtype='object')

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk.stem
from nltk.corpus import stopwords
english_stemmer=nltk.stem.SnowballStemmer('english')
stwe=stopwords.words('english')
class EnglishStemmedTfidfVectorizer(TfidfVectorizer):#EnglishtemmedTfidVectorizer hérite de TfidVectorizer
    def build_analyzer(self):
        analyzer = super(EnglishStemmedTfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))


In [8]:
ftvectorizer = EnglishStemmedTfidfVectorizer(max_df=0.6, stop_words=stwe,decode_error='ignore')
#ne retient que les termes qui apparaissent dans au plus 60% des documents
X = ftvectorizer.fit_transform(cyberpunk["story_text"])
dico=ftvectorizer.vocabulary_
Ps=len(dico)
print("taille du vocabulaire après suppression des stop_words : ",Ps)


taille du vocabulaire après suppression des stop_words :  43502


In [9]:
ftvectorizer = EnglishStemmedTfidfVectorizer(min_df=0.4, stop_words=stwe,decode_error='ignore')
X = ftvectorizer.fit_transform(cyberpunk["story_text"])
dico=ftvectorizer.vocabulary_
Ps=len(dico)
print("taille du vocabulaire après suppression des stop_words : ",Ps)

taille du vocabulaire après suppression des stop_words :  343


In [10]:
len(y.unique())

NameError: name 'y' is not defined

In [ ]:

from sklearn.cluster import KMeans
clust=KMeans(n_clusters=12,n_init=5,verbose=1)
Ckm=clust.fit_predict(X) # pour obtenir les labels (n° classe) de chaque élément de la base
Xd=clust.transform(X) #pour avoir les distances de chaque élément aux centres des clusters


In [ ]:
Ckm

In [ ]:
for i,y in enumerate(y_fit):
    if y == 2:
        print(y,Ckm[i])

In [ ]:
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
J=np.zeros((12,))
for K in range(1,12):
    print('************************************')
    print('K = ',K)
    clust=KMeans(n_clusters=K)
    Ckm=clust.fit_predict(X)
    J[K]=clust.inertia_
    
f, axarr = plt.subplots(2, sharex=True)

axarr[0].plot(np.arange(1,12),J[1:12])  
axarr[0].set_title('évolution du critère')
axarr[1].plot(np.arange(2,12),(J[1:11]-J[2:12])/J[1:11])
axarr[1].set_title('évolution relative du critère')   

In [ ]:
X.toarray()

In [ ]:
X = X.toarray()
y = cyberpunk["year_publication"]

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y_fit = le.transform(y)

In [ ]:
y_fit

In [ ]:
seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, y_fit, test_size=0.20, random_state=seed)


In [ ]:
clf = LogisticRegression(penalty="l1",solver="liblinear").fit(X_train, y_train)
clf.score(X_test, y_test)

On n'arrive pas trop à discrimner (20 % mieux que 1/12 mais bon)